# DBT Tutorial 101 - Recommender System Lab

This notebook demonstrates two approaches to building a recommendation system using the data transformed by dbt:

1. **Weighted Hybrid Recommender**: Combines Collaborative Filtering, Market Basket Analysis, and Content-Based Filtering.
2. **Gradient Boosting Ranker**: A 'Learning to Rank' approach that predicts the probability of purchase based on context.

## Prerequisites
Ensure you have run `dbt run` so that `raw.fact_order_items` and `raw.fact_orders` exist in your local Postgres.
    3. **Neural Collaborative Filtering**: Deep Learning approach using PyTorch.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import logging
import sys

# Setup logging to show progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# Database Configuration
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "postgres",
    "user": "postgres",
    "password": "mysecretpassword",
    "schema": "raw"
}

def get_db_connection():
    conn_str = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    return create_engine(conn_str)

engine = get_db_connection()
print("Database connection established.")

## Part 1: Weighted Hybrid Recommender

We will load the `fact_order_items` table and build 4 distinct recommendation models:
1. **Popularity**: Baseline.
2. **Collaborative Filtering (CF)**: Item-Item Cosine Similarity.
3. **Association Rules**: Apriori algorithm for Market Basket Analysis.
4. **Content-Based**: Metadata similarity (Price, Type).

In [ ]:
# Load Interaction Data
query = f"SELECT * FROM {DB_CONFIG['schema']}.fact_order_items"
df_items = pd.read_sql(query, engine)
print(f"Loaded {len(df_items)} rows.")
df_items.head()

In [ ]:
class WeightedHybridRecommender:
    def __init__(self, df, weights={'cf': 0.4, 'content': 0.3, 'rules': 0.2, 'pop': 0.1}):
        self.df = df
        self.weights = weights
        
        # Models
        self.basket_encoded = None
        self.item_sim_df = None # CF
        self.content_sim_df = None # Content
        self.rules = None # Rules
        self.global_pop_score = None # Popularity
        
        self._train_models()
        
    def _train_models(self):
        print("Training component models...")
        
        # 1. Global Popularity
        pop_counts = self.df['product_name'].value_counts()
        self.global_pop_score = (pop_counts - pop_counts.min()) / (pop_counts.max() - pop_counts.min())
        
        # 2. Collaborative Filtering (Item-Item)
        basket = self.df.groupby(['order_id', 'product_name'])['product_name'].count().unstack().reset_index().fillna(0).set_index('order_id')
        self.basket_encoded = basket.map(lambda x: 1 if x >= 1 else 0)
        
        item_user_matrix = self.basket_encoded.T
        sim_matrix = cosine_similarity(item_user_matrix)
        self.item_sim_df = pd.DataFrame(sim_matrix, index=self.basket_encoded.columns, columns=self.basket_encoded.columns)
        
        # 3. Association Rules
        frequent_itemsets = apriori(self.basket_encoded.astype(bool), min_support=0.005, use_colnames=True)
        if not frequent_itemsets.empty:
            self.rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
        else:
            self.rules = pd.DataFrame()
            
        # 4. Content Based
        product_features = self.df[['product_name', 'product_price', 'is_food_item', 'is_drink_item']].drop_duplicates('product_name').set_index('product_name')
        product_features['product_price'] = (product_features['product_price'] - product_features['product_price'].mean()) / product_features['product_price'].std()
        content_sim = cosine_similarity(product_features.fillna(0))
        self.content_sim_df = pd.DataFrame(content_sim, index=product_features.index, columns=product_features.index)
        
        print("Training complete.")

    def recommend(self, product_name=None, basket_items=None, top_n=5):
        all_products = self.global_pop_score.index.tolist()
        scores = {p: 0.0 for p in all_products}
        
        # 1. Popularity
        w_pop = self.weights.get('pop', 0.1)
        for p, score in self.global_pop_score.items():
            scores[p] += score * w_pop
            
        # 2. CF
        if product_name and product_name in self.item_sim_df.index:
            w_cf = self.weights.get('cf', 0.4)
            sim_scores = self.item_sim_df[product_name]
            for p, score in sim_scores.items():
                scores[p] += score * w_cf
                
        # 3. Content
        if product_name and product_name in self.content_sim_df.index:
            w_content = self.weights.get('content', 0.3)
            sim_scores = self.content_sim_df[product_name]
            for p, score in sim_scores.items():
                scores[p] += score * w_content
                
        # 4. Rules
        if basket_items and not self.rules.empty:
            w_rules = self.weights.get('rules', 0.2)
            basket_set = set(basket_items)
            relevant_rules = self.rules[self.rules['antecedents'].apply(lambda x: x.issubset(basket_set))]
            for _, row in relevant_rules.iterrows():
                for item in list(row['consequents']):
                    scores[item] += row['confidence'] * w_rules

        # Filter self
        exclude = set()
        if product_name: exclude.add(product_name)
        if basket_items: exclude.update(basket_items)
        
        final_scores = {k: v for k, v in scores.items() if k not in exclude}
        ranked = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
        return ranked[:top_n]

In [ ]:
# Initialize User-Defined Weights
weights = {'cf': 0.4, 'content': 0.2, 'rules': 0.3, 'pop': 0.1}
recommender = WeightedHybridRecommender(df_items, weights)

In [ ]:
# TEST 1: Viewing 'doctor stew'
print("Recommendations for 'doctor stew':")
recs = recommender.recommend(product_name="doctor stew", basket_items=[])
pd.DataFrame(recs, columns=['Product', 'Score'])

In [ ]:
# TEST 2: Basket contains 'mel-bun'
print("Recommendations for Basket=['mel-bun']:")
recs = recommender.recommend(product_name=None, basket_items=['mel-bun'])
pd.DataFrame(recs, columns=['Product', 'Score'])

## Part 2: Gradient Boosting Ranker

In this section, we treat recommendation as a binary classification problem: "Will the user buy this item given the context?"

We need to generate **Negative Samples** (items the user *didn't* buy) to train the model.

In [ ]:
print("Loading data for Grading Boosting...")
# Fetch Order Items + Context (Time)
query = f"""
    SELECT 
        i.order_id,
        i.product_id,
        i.product_price,
        i.is_food_item,
        i.is_drink_item,
        o.ordered_at
    FROM {DB_CONFIG['schema']}.fact_order_items i
    JOIN {DB_CONFIG['schema']}.fact_orders o ON i.order_id = o.order_id
    LIMIT 20000
"""
df_pos = pd.read_sql(query, engine)
df_pos['target'] = 1

print(f"Loaded {len(df_pos)} positive samples.")

In [ ]:
# Generate Negative Samples
unique_products = df_pos[['product_id', 'product_price', 'is_food_item', 'is_drink_item']].drop_duplicates()
all_product_ids = unique_products['product_id'].values
purchased_map = df_pos.groupby('order_id')['product_id'].apply(set).to_dict()
orders_unique = df_pos[['order_id', 'ordered_at']].drop_duplicates()

neg_samples = []
for _, row in orders_unique.iterrows():
    oid = row['order_id']
    bought_items = purchased_map.get(oid, set())
    candidates = [p for p in all_product_ids if p not in bought_items]
    
    if candidates:
        chosen_neg = np.random.choice(candidates, size=min(len(candidates), 2), replace=False)
        for neg_pid in chosen_neg:
            prod_info = unique_products[unique_products['product_id'] == neg_pid].iloc[0]
            neg_samples.append({
                'order_id': oid,
                'product_id': neg_pid,
                'product_price': prod_info['product_price'],
                'is_food_item': prod_info['is_food_item'],
                'is_drink_item': prod_info['is_drink_item'],
                'ordered_at': row['ordered_at'],
                'target': 0
            })
            
df_neg = pd.DataFrame(neg_samples)
df_full = pd.concat([df_pos, df_neg], ignore_index=True)
print(f"Total dataset size: {len(df_full)}")

In [ ]:
# Feature Engineering
df_full['ordered_at'] = pd.to_datetime(df_full['ordered_at'])
df_full['hour_of_day'] = df_full['ordered_at'].dt.hour
df_full['day_of_week'] = df_full['ordered_at'].dt.dayofweek
df_full['is_weekend'] = df_full['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

feature_cols = ['product_price', 'is_food_item', 'is_drink_item', 'hour_of_day', 'day_of_week', 'is_weekend']
X = df_full[feature_cols]
y = df_full['target']

# Handle types
X['is_food_item'] = X['is_food_item'].astype(int)
X['is_drink_item'] = X['is_drink_item'].astype(int)

In [ ]:
# Train Gradient Boosting Classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Model Accuracy: {acc:.4f}")

In [ ]:
# Feature Importance Visualization
importance = model.feature_importances_
indices = np.argsort(importance)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importance - Gradient Boosting")
plt.bar(range(X.shape[1]), importance[indices], align="center")
plt.xticks(range(X.shape[1]), [feature_cols[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()

## Part 3: Neural Collaborative Filtering (Deep Learning)

We will use **PyTorch** to build a Neural Collaborative Filtering (NCF) model. 
This model learns latent vector representations (embeddings) for Orders and Products to predict the likelihood of a purchase.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# PyTorch Dataset
class OrderItemDataset(Dataset):
    def __init__(self, user_ids, item_ids, targets):
        self.user_ids = torch.tensor(user_ids, dtype=torch.long)
        self.item_ids = torch.tensor(item_ids, dtype=torch.long)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.targets[idx]

# Neural Network Model
class NCFModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(NCFModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Simple MLP
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user_idx, item_idx):
        u_emb = self.user_embedding(user_idx)
        i_emb = self.item_embedding(item_idx)
        x = torch.cat([u_emb, i_emb], dim=1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
# Data Preparation for Deep Learning

# Re-load fresh data
query = f"SELECT order_id, product_name FROM {DB_CONFIG['schema']}.fact_order_items"
df_dl = pd.read_sql(query, engine)

# Encode IDs
order_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df_dl['order_idx'] = order_encoder.fit_transform(df_dl['order_id'])
df_dl['item_idx'] = item_encoder.fit_transform(df_dl['product_name'])

num_orders = len(order_encoder.classes_)
num_items = len(item_encoder.classes_)
print(f"Deep Learning Input: {num_orders} Orders, {num_items} Items")

# Create Samples (Positive + Negative)
user_ids = df_dl['order_idx'].values
item_ids = df_dl['item_idx'].values
targets = np.ones(len(df_dl))

# Negative Sampling (1:1)
neg_user_ids = np.random.randint(0, num_orders, len(df_dl))
neg_item_ids = np.random.randint(0, num_items, len(df_dl))
neg_targets = np.zeros(len(df_dl))

all_users = np.concatenate([user_ids, neg_user_ids])
all_items = np.concatenate([item_ids, neg_item_ids])
all_targets = np.concatenate([targets, neg_targets])

In [ ]:
# Train Loop
dataset = OrderItemDataset(all_users, all_items, all_targets)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

model_ncf = NCFModel(num_orders, num_items, embedding_dim=32)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_ncf.parameters(), lr=0.001)

print("Training Neural Network...")
epochs = 5
for epoch in range(epochs):
    total_loss = 0
    for u, i, t in dataloader:
        optimizer.zero_grad()
        predictions = model_ncf(u, i).squeeze()
        loss = criterion(predictions, t)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(dataloader):.4f}")

In [ ]:
# Visualize Learned Embeddings Similarity
model_ncf.eval()
with torch.no_grad():
    item_embeddings = model_ncf.item_embedding.weight.data.numpy()

def get_neural_recs(product_name, top_n=5):
    if product_name not in item_encoder.classes_:
        return []
    
    idx = item_encoder.transform([product_name])[0]
    target_emb = item_embeddings[idx]
    
    # Cosine Sim
    norms = np.linalg.norm(item_embeddings, axis=1)
    dot_products = np.dot(item_embeddings, target_emb)
    sims = dot_products / (norms * np.linalg.norm(target_emb))
    
    top_indices = np.argsort(sims)[::-1][1:top_n+1]
    return [(item_encoder.inverse_transform([i])[0], sims[i]) for i in top_indices]

print("Neural Recommendations for 'doctor stew':")
for item, score in get_neural_recs('doctor stew'):
    print(f"{item} (Sim: {score:.4f})")